In [14]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold

In [25]:
df = pd.read_csv("./sample.csv")
df.head()

,female,read,write,math,hon,femalexmath
0,0,57,52,41,0,0
1,1,68,59,53,0,53
2,0,44,33,54,0,0
3,0,63,44,47,0,0
4,0,47,52,57,0,0


In [92]:
class my_logistic_reg:
  def predict(self, X, w):
      '''Bias col must be added before calling this fun'''
      y_pred = X @ w
      y_pred = 1/(1 + np.exp(-y_pred)) # apply sigmoid fun
      return y_pred
      
  def fit(self, X, y, lr = 0.01,max_epochs=1000):
      
      X = np.c_[np.ones((X.shape[0], 1)), X] # add bias col where n_samples = X.shape[0]
      y = np.array(y).reshape(-1,1)
      precision = 0.00000000001
      change_in_w = 1
      w = np.random.rand(X.shape[1],1)
      err_lst=[]
      iter = 0

      while( iter <= max_epochs):
          prev_w = w
          
          y_pred = self.predict(X,w)

          dE = X.T @ (y_pred - y)
          
          w = w - (lr*dE)
          iter += 1
          err_lst.append(np.abs(np.sum(y-y_pred)))

      return w, err_lst
 

In [62]:
def find_accuracy(y_test, y_pred):
  maxx = len(y_test)
  c=0
  for i in range(maxx):
    if(y_test[i] == y_pred[i]):
      c+= 1
  
  return c/maxx

#Using inbuilt lib

In [65]:
kf = KFold(n_splits=5)

lst=[]
X = df.iloc[:, :4]
y = df.iloc[:, 4]
reg_obj = LogisticRegression()

for train_index, test_index in kf.split(X):
  X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :] 
  y_train, y_test = y[train_index], y[test_index]
  
  reg_obj.fit(X_train,y_train)

  y_pred= reg_obj.predict(X_test)
  acc = find_accuracy(list(y_test), list(y_pred))
  lst.append(acc)


avg_accuracy = sum(lst)/5
avg_accuracy

0.9949999999999999

#using custom class

In [96]:
reg_obj = my_logistic_reg()
lst = []
for train_index, test_index in kf.split(X):
  X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :] 
  y_train, y_test = y[train_index], y[test_index]
  
  w, err_lst = reg_obj.fit(X_train, y_train, lr = 0.01,max_epochs=8000) 

  X_test = np.c_[np.ones((X_test.shape[0], 1)), X_test] # add bias col in x before predict 
  y_pred= reg_obj.predict(X_test, w)

  acc = find_accuracy(list(y_test), list(y_pred))
  lst.append(acc)

avg_accuracy = sum(lst)/5
avg_accuracy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in exp
  """


0.7000000000000001

I started with 1000 epochs, but got 0.58244 avg_accuracy. I was keep increasing to check its effect on avg_accuracy, finally at lr = 0.01,max_epochs=8000 I got 0.7 avg_accuracy.